Run this cell to define support functions

In [ ]:
import sys
import os
import math
import numpy as np
from comtrade import Comtrade

kVLNbase = 230.0 / math.sqrt(3.0)
MVAbase = 100.0
kAbase = MVAbase / kVLNbase / 3.0

def scale_factor(lbl):
  if 'P' in lbl:
    return 1.0 / MVAbase
  elif 'Q' in lbl:
    return 1.0 / MVAbase
  elif 'I' in lbl:
    return 1.0 / kAbase / math.sqrt(2.0)
  elif 'Vrms' in lbl:
    return 1.0
  elif 'V' in lbl:
    return 1.0 / kVLNbase / math.sqrt(2.0)
  return 1.0

# load all the analog channels from each case into dictionaries of numpy arrays. Expecting:
#   1..3 = Va..Vc
#   4..6 = Ia..Ic
#   7 = Vrms
#   8 = P
#   9 = Q
#   10 = F
def load_channels(comtrade_path):
  rec = Comtrade ()
  rec.load (comtrade_path + '.cfg', comtrade_path + '.dat')
  t = np.array(rec.time)

  channels = {}
  units = {}
  channels['t'] = t
  print ('{:d} channels ({:d} points) read from {:s}.cfg:'.format (rec.analog_count, len(t), comtrade_path))
  for i in range(rec.analog_count):
    lbl = rec.analog_channel_ids[i].strip()
    # for PSCAD naming convention, truncate the channel at first colon, if one exists
    idx = lbl.find(':')
    if idx >= 0:
      lbl = lbl[0:idx]
    ch_config = rec.cfg.analog_channels[i]
    scale = 1.0
    if ch_config.pors.upper() == 'P':
      scale = ch_config.secondary / ch_config.primary
    elif ch_config.pors.upper() == 'S':
      scale = ch_config.primary / ch_config.secondary
    print ('  "{:s}" [{:s}] scale={:.6e}'.format(lbl, ch_config.uu, scale))
    channels[lbl] = scale * np.array (rec.analog[i])
    units[lbl] = ch_config.uu

  return channels, units

Run this next cell to enable [Matplotlib](https://matplotlib.org/)

In [ ]:
import matplotlib.pyplot as plt

def setup_plot_options():
  plt.rcParams['savefig.directory'] = os.getcwd()
  lsize = 12
  plt.rc('font', family='serif')
  plt.rc('xtick', labelsize=lsize)
  plt.rc('ytick', labelsize=lsize)
  plt.rc('axes', labelsize=lsize)
  plt.rc('legend', fontsize=lsize)

def show_case_plot(channels, units, case_title):
  t = channels['t']
  fig, ax = plt.subplots(5, 1, sharex = 'col', figsize=(8,6), constrained_layout=True)
  fig.suptitle ('Case: ' + case_title)
  for lbl in ['VA', 'VB', 'VC']:
    ax[0].plot (t, scale_factor(lbl) * channels[lbl], label=lbl)
    ax[0].set_ylabel ('v(t) [pu]')
  for lbl in ['IA', 'IB', 'IC']:
    ax[1].plot (t, scale_factor(lbl) * channels[lbl], label=lbl)
    ax[1].set_ylabel ('i(t) [pu]')
  for lbl in ['Vrms']:
    ax[2].plot (t, scale_factor(lbl) * channels[lbl], label=lbl)
    ax[2].set_ylabel ('V [pu]')
  for lbl in ['P', 'Q']:
    ax[3].plot (t, scale_factor(lbl) * channels[lbl], label=lbl)
    ax[3].set_ylabel ('P, Q [pu]')
  for lbl in ['F']:
    ax[4].plot (t, scale_factor(lbl) * channels[lbl], label=lbl)
    ax[4].set_ylabel ('F [Hz]')
  for i in range(5):
    ax[i].grid()
    ax[i].legend()
#    ax[i].set_xlim (t[0], t[-1])
    ax[i].set_xlim (0.75, 1.75)
  ax[4].set_xlabel ('seconds')
  plt.show()
#  plt.close()

def show_comparison_plot (dm, avm, units):
  fig, ax = plt.subplots(4, 1, sharex = 'col', figsize=(8,6), constrained_layout=True)
  fig.suptitle ('Comparing Average and Switching Models')

  channel_labels = ['Vrms', 'P', 'Q', 'F']
  y_labels = ['Vrms [pu]', 'P [pu]', 'Q [pu]', 'F [Hz]']
  x_ticks = [0.75, 1.00, 1.25, 1.50, 1.75]

  for i in range(4):
    lbl = channel_labels[i]
    ax[i].set_ylabel (y_labels[i])
    ax[i].plot (dm['t'], scale_factor(lbl) * dm[lbl], label='DM')
    ax[i].plot (avm['t'], scale_factor(lbl) * avm[lbl], label='AVM')
    ax[i].set_xticks (x_ticks)
    ax[i].set_xlim (x_ticks[0], x_ticks[-1])
    ax[i].grid()
    ax[i].legend()
  ax[3].set_xlabel ('Time [s]')
  plt.show()
#  plt.close()

%matplotlib widget

setup_plot_options()

Run the next cell for PSCAD results import

In [ ]:
# set the session_path to match location of your unzipped sample cases
session_path = 'c:/temp/i2x/pscad/'
dm_path = os.path.join (session_path, 'Solar3dm.if18_x86/rank_00001/Run_00001/DM')
avm_path = os.path.join (session_path, 'Solar3avm.if18_x86/rank_00001/Run_00001/AVM')

dm_channels, dm_units = load_channels (dm_path)
avm_channels, avm_units = load_channels (avm_path)

Run the next cell for EMTP results import

In [ ]:
# set the session_path to match location of your unzipped sample cases
session_path = 'c:/temp/i2x/emtp/'
dm_path = os.path.join (session_path, 'dm')
avm_path = os.path.join (session_path, 'avm')

dm_channels, dm_units = load_channels (dm_path)
avm_channels, avm_units = load_channels (avm_path)

Run the next series of cells to plot the results

In [ ]:
show_case_plot (dm_channels, dm_units, 'Switching Model')

In [ ]:
show_case_plot (avm_channels, avm_units, 'Average Model')

In [ ]:
show_comparison_plot (dm=dm_channels, avm=avm_channels, units=dm_units)

Run the next cell to compare harmonics on a grid current

In [ ]:
from scipy.fft import fft, fftfreq

def grid_current_window (ch, lbl, t1, t2):
    n1=min(np.argwhere(ch['t']>=t1))[0]
    n2=max(np.argwhere(ch['t']<=t2))[0]
    t=ch['t'][n1:n2]
    i=ch[lbl][n1:n2]
    return t, i

def get_spectrum (t, y, lbl=None):
  N = len(t)
  dt = t[1] - t[0]
  yf = fft(y)
  f = fftfreq(N, dt)[:N//2]
  m = np.abs(yf[0:N//2])
  # scale fft "signal processing" magnitude by 2/N to get "power system" magnitude
  m *= 2.0/N
  if lbl is not None:
    print (lbl)
    print ('  FFT on {:d} points at dt={:.6f}'.format (N, dt))
    m1peak = np.max(m)
    m1rms = m1peak / np.sqrt(2.0)
    rms = np.sqrt (0.5 * np.mean (np.sum (m*m)))
    thd = np.sqrt (rms*rms - m1rms*m1rms) / m1rms
    print ('  Fundamental Peak = {:.4f}'.format (m1peak))
    print ('  Fundamental RMS =  {:.4f}'.format (m1rms))
    print ('  Total RMS =        {:.4f}'.format (rms))
    print ('  Total Distortion = {:.4f}%'.format (thd*100.0))
  return f, m * 2.0 / N
    
def fft_plot (t_dm, y_dm, t_avm, y_avm, f_dm, m_dm, f_avm, m_avm, chan_label, bSemiLog=False):
  fig, ax = plt.subplots(2, 1, figsize=(8,6), constrained_layout=True)
  fig.suptitle ('Harmonic analysis of {:s}'.format (chan_label))
  f_ticks = np.linspace (0, 1000, 11)

  ax[0].plot (t_dm, y_dm, label='DM')
  ax[0].plot (t_avm, y_avm, label='AVM')
  ax[0].set_xlabel ('Time [s]')
  if bSemiLog:
    ax[1].semilogy (f_dm, m_dm, label='DM')
    ax[1].semilogy (f_avm, m_avm, label='AVM')
  else:
    ax[1].plot (f_dm, m_dm, label='DM')
    ax[1].plot (f_avm, m_avm, label='AVM')
  ax[1].set_xlabel ('Frequency [Hz]')
  ax[1].set_xticks (f_ticks)
  ax[1].set_xlim (f_ticks[0], f_ticks[-1])
  for i in range(2):
    ax[i].grid()
    ax[i].legend()

  plt.show()

t_start = 0.5
t_end = t_start + 0.10

t_dm, i_dm = grid_current_window (dm_channels, 'IA', t_start, t_end)
t_avm, i_avm = grid_current_window (avm_channels, 'IA', t_start, t_end)

f_dm, m_dm = get_spectrum (t_dm, i_dm, 'Switching Model')
f_avm, m_avm = get_spectrum (t_avm, i_avm, 'Average Model')

fft_plot (t_dm, i_dm, t_avm, i_avm, f_dm, m_dm, f_avm, m_avm, 'IA', bSemiLog=True)